In [10]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import torchvision.transforms as transforms
import numpy as np
import matplotlib.pyplot as plt
import random


In [11]:
#Load dataset CIFAR10
transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])
train_dataset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
test_dataset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)

train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=100, shuffle=True)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=100, shuffle=False)

classes = ('plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

In [12]:
test_loader.dataset.data.shape

(10000, 32, 32, 3)

In [13]:
#Device configurarion
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
torch.cuda.is_available()

True

In [14]:
#Implement a CNN
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()

        # Bloque convolucional 1
        self.conv1 = nn.Conv2d(3, 6, 5)       # in_channels=3, out_channels=6, kernel_size=5, padding=2
        self.bn1 = nn.BatchNorm2d(6)

        # Bloque convolucional 2
        self.pool = nn.MaxPool2d(2, 2)        # kernel_size=2, stride=2
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.bn2 = nn.BatchNorm2d(16)

        # Fully Connected Layers
        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.bn3 = nn.BatchNorm1d(120)
        self.fc2 = nn.Linear(120, 84)
        self.bn4 = nn.BatchNorm1d(84)
        self.fc3 = nn.Linear(84, 10)

        self.con_dropout = nn.Dropout(0.1)
        self.flt_dropout = nn.Dropout(0.3)
        self.relu = nn.ReLU()

    def forward(self, x):
        # CONV + BN + ReLU + Pool + Drop
        x = self.conv1(x)
        x = self.bn1(x)
        x = self.relu(x)
        x = self.pool(x)
        x = self.con_dropout(x)

        x = self.conv2(x)
        x = self.bn2(x)
        x = self.relu(x)
        x = self.pool(x)
        x = self.con_dropout(x)

        # Flatten & FC
        x = x.view(-1, 16 * 5 * 5)
        x = self.fc1(x)
        x = self.bn3(x)
        x = self.relu(x)
        x = self.flt_dropout(x)

        x = self.fc2(x)
        x = self.bn4(x)
        x = self.relu(x)
        x = self.flt_dropout(x)

        x = self.fc3(x)
        return x

In [15]:
# --- Modelo LeNet simple ---
class LeNet(nn.Module):
    def __init__(self, num_classes=10):
        super().__init__()
        self.conv1 = nn.Conv2d(3, 6, kernel_size=5)   # 3x32x32 -> 6x28x28
        self.pool  = nn.MaxPool2d(2, 2)               # -> 6x14x14
        self.conv2 = nn.Conv2d(6, 16, kernel_size=5)  # -> 16x10x10
        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, num_classes)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(x.size(0), -1)  # flatten
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x


In [16]:
#Model and Hyper parameters

# Instantiate the CNN model and move it to the configured device.
model = LeNet().to(device)

# Define the loss function and optimizer.
# CrossEntropyLoss is commonly used for classification tasks.
criterion = nn.CrossEntropyLoss(label_smoothing=0.05)
# Adam optimizer is an effective optimization algorithm.
optimizer = torch.optim.AdamW(model.parameters(), lr=0.001, weight_decay=1e-4)

# Training loop
num_epochs = 30  # Number of training epochs

# Scheduler de tasa de aprendizaje
scheduler = torch.optim.lr_scheduler.OneCycleLR(
    optimizer,
    max_lr=0.01,
    steps_per_epoch=len(train_loader),
    epochs=num_epochs
)


In [17]:
#Model and Hyper parameters

# Instantiate the CNN model and move it to the configured device.
model = LeNet().to(device)

# Define the loss function and optimizer.
# CrossEntropyLoss is commonly used for classification tasks.
criterion = nn.CrossEntropyLoss()
# Adam optimizer is an effective optimization algorithm.
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

# Training loop
num_epochs = 30  # Number of training epochs


In [43]:
def refresh_small_weights(model, small_thr=1e-4, frac=0.5, scale=0.1, exclude_norm=True):
    total = 0
    with torch.no_grad():
        for name, p in model.named_parameters():
            if not p.requires_grad:
                continue
            if exclude_norm and (name.endswith(".bias") or "bn" in name.lower() or "norm" in name.lower()):
                continue
            small = p.abs() < small_thr
            if not small.any():
                continue
            pick = torch.rand_like(p) < frac
            mask = small & pick
            k = mask.sum().item()
            if k == 0:
                continue
            sigma = (p.std().item() + 1e-8) * scale
            p[mask] = torch.randn_like(p[mask]) * sigma
            total += k
    return total


In [45]:
for epoch in range(num_epochs):
    # ---- TRAIN ----
    model.train()
    running_loss_train = 0.0
    correct_train = 0
    total_train = 0

    for i, (images, labels) in enumerate(train_loader):
        images = images.to(device)
        labels = labels.to(device)

        outputs = model(images)
        loss = criterion(outputs, labels)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        running_loss_train += loss.item() * labels.size(0)  # acumula total loss
        _, predicted = torch.max(outputs.data, 1)
        total_train += labels.size(0)
        correct_train += (predicted == labels).sum().item()

        # --- Refrescar pesos pequeños ---
        if (i + 1) % 100 == 0:
            n = refresh_small_weights(model, small_thr=1e-2, frac=0.5, scale=0.1)
            print(f"[refresh step {i+1}] total reset: {n}")


    avg_train_loss = running_loss_train / total_train
    avg_train_acc = 100 * correct_train / total_train

    # ---- TEST ----
    model.eval()
    running_loss_test = 0.0
    correct_test = 0
    total_test = 0

    with torch.no_grad():
        for images, labels in test_loader:
            images = images.to(device)
            labels = labels.to(device)

            outputs = model(images)
            loss = criterion(outputs, labels)

            running_loss_test += loss.item() * labels.size(0)
            _, predicted = torch.max(outputs.data, 1)
            total_test += labels.size(0)
            correct_test += (predicted == labels).sum().item()

    avg_test_loss = running_loss_test / total_test
    avg_test_acc = 100 * correct_test / total_test

    # ---- PRINT ----
    print(f"Epoch [{epoch+1}/{num_epochs}] "
          f"Train Loss: {avg_train_loss:.4f}, Train Acc: {avg_train_acc:.2f}% | "
          f"Test Loss: {avg_test_loss:.4f}, Test Acc: {avg_test_acc:.2f}%")

print('Finished Training')

[refresh step 100] total reset: 1914
[refresh step 200] total reset: 1350
[refresh step 300] total reset: 922
[refresh step 400] total reset: 698
[refresh step 500] total reset: 568
Epoch [1/30] Train Loss: 0.2993, Train Acc: 89.06% | Test Loss: 2.5003, Test Acc: 59.04%
[refresh step 100] total reset: 482
[refresh step 200] total reset: 364
[refresh step 300] total reset: 295
[refresh step 400] total reset: 303
[refresh step 500] total reset: 286
Epoch [2/30] Train Loss: 0.2899, Train Acc: 89.55% | Test Loss: 2.5826, Test Acc: 58.56%
[refresh step 100] total reset: 232
[refresh step 200] total reset: 221


KeyboardInterrupt: 

In [ ]:
        # --- Refrescar pesos pequeños ---
        if (i+1) % 20 == 0:
            with torch.no_grad():
                for param in model.parameters():
                    small_mask = param.abs() < 1e-7  # pesos pequeños
                    # De esos pequeños, seleccionar aleatoriamente un % (aquí 50%)
                    rand_mask = torch.rand_like(param) < 0.5 #Probabilidadd de reseteo
                    # Máscara final: pequeños Y seleccionados aleatoriamente
                    final_mask = small_mask & rand_mask
                    # Reemplazar solo esos
                    param[final_mask] = torch.randn_like(param[final_mask]) * 0.1
